# Homework 10
#### Course Notes
**Language Models:** https://github.com/rjenki/BIOS512/tree/main/lecture17  
**Unix:** https://github.com/rjenki/BIOS512/tree/main/lecture18  
**Docker:** https://github.com/rjenki/BIOS512/tree/main/lecture19

## Question 1
#### Make a language model that uses ngrams and allows the user to specify start words, but uses a random start if one is not specified.

#### a) Make a function to tokenize the text.

In [10]:
install.packages("tokenizers")
library(httr)
library(tokenizers)

tokenize_text <- function(text) {
    tokenizers::tokenize_words(text, lowercase = TRUE, strip_punct = TRUE)[[1]]
}


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘SnowballC’




#### b) Make a function generate keys for ngrams.

In [ ]:
install.packages(tokenizers)
library(httr)
library(tokenizers)

tokenize_text <- function(text) {
    tokenizers::tokenize_words(text, lowercase = TRUE, strip_punct = TRUE)[[1]]
}

key_from <- function(ngram, sep = "\x1f") {
    paste(ngram, collapse = sep)
}

#### c) Make a function to build an ngram table.

In [3]:
install.packages(tokenizers)
library(httr)
library(tokenizers)

tokenize_text <- function(text) {
    tokenizers::tokenize_words(text, lowercase = TRUE, strip_punct = TRUE)[[1]]
}

key_from <- function(ngram, sep = "\x1f") {
    paste(ngram, collapse = sep)
}


build_ngram_table <- function(tokens, n, sep = "\x1f") {
    if (length(tokens) < n) return(new.env(parent = emptyenv()))

    tbl <- new.env(parent = emptyenv())

    for (i in seq_len(length(tokens) - n + 1L)) {

        ngram <- tokens[i:(i + n - 2L)]

        next_word <- tokens[i + n - 1L]


        key <- key_from(ngram, sep)


        counts <- if (!is.null(tbl[[key]])) tbl[[key]] else integer(0)
        if (next_word %in% names(counts)) {
            counts[[next_word]] <- counts[[next_word]] + 1L
        } else {
            counts[[next_word]] <- 1L
        }
        tbl[[key]] <- counts
    }

    tbl
}

ERROR: Error in library(tokenizers): there is no package called ‘tokenizers’


#### d) Function to digest the text.

In [8]:
digest_text <- function(text, n) {
    tokens <- tokenize_text(text)
    build_ngram_table(tokens, n)
}

#### e) Function to digest the url.

In [7]:
digest_url <- function(url, n) {
    res <- httr::GET(url)
    txt <- httr::content(res, as = "text", encoding = "UTF-8")
    digest_text(txt, n)
}

#### f) Function that gives random start.

In [6]:
random_start <- function(tbl, sep = "\x1f") {
    keys <- ls(envir = tbl, all.names = TRUE)
    if (length(keys) == 0) stop("No n-grams available. Digest text first.")
    picked <- sample(keys, 1)
    strsplit(picked, sep, fixed = TRUE)[[1]]
}

#### g) Function to predict the next word.

In [4]:
predict_next_word <- function(tbl, ngram, sep = "\x1f") {
    key <- paste(ngram, collapse = sep)
    counts <- if (!is.null(tbl[[key]])) tbl[[key]] else integer(0)
    if (length(counts) == 0) return(NA_character_)
    sample(names(counts), size = 1, prob = as.numeric(counts))
}


#### h) Function that puts everything together. Specify that if the user does not give a start word, then the random start will be used.

In [5]:
make_ngram_generator <- function(tbl, n, sep = "\x1f") {
    force(tbl)
    n <- as.integer(n)
    force(sep)
 function(start_words = NULL, length = 10L) {
  if (is.null(start_words) || length(start_words) != n - 1L) {
            start_words <- random_start(tbl, sep = sep)
        }

        word_sequence <- start_words
  for (i in seq_len(max(0L, length - length(start_words)))) {
            ngram <- tail(word_sequence, n - 1L)
            next_word <- predict_next_word(tbl, ngram, sep = sep)
            if (is.na(next_word)) break
            word_sequence <- c(word_sequence, next_word)
        }

        paste(word_sequence, collapse = " ")
    }
}

## Question 2
#### For this question, set `seed=2025`.
#### a) Test your model using a text file of [Grimm's Fairy Tails](https://www.gutenberg.org/cache/epub/2591/pg2591.txt)
#### i) Using n=3, with the start word(s) "the king", with length=15.
#### ii) Using n=3, with no start word, with length=15.

In [15]:
install.packages("tokenizers")
library(httr)
library(tokenizers)

url <- "https://www.gutenberg.org/cache/epub/2591/pg2591.txt"

tokenize_text <- function(text) {
    tokenizers::tokenize_words(text, lowercase = TRUE, strip_punct = TRUE)[[1]]
}

key_from <- function(ngram, sep = "\x1f") {
    paste(ngram, collapse = sep)
}


build_ngram_table <- function(tokens, n, sep = "\x1f") {
    if (length(tokens) < n) return(new.env(parent = emptyenv()))

    tbl <- new.env(parent = emptyenv())

    for (i in seq_len(length(tokens) - n + 1L)) {

        ngram <- tokens[i:(i + n - 2L)]

        next_word <- tokens[i + n - 1L]


        key <- key_from(ngram, sep)


        counts <- if (!is.null(tbl[[key]])) tbl[[key]] else integer(0)
        if (next_word %in% names(counts)) {
            counts[[next_word]] <- counts[[next_word]] + 1L
        } else {
            counts[[next_word]] <- 1L
        }
        tbl[[key]] <- counts
    }

    tbl
}
digest_text <- function(text, n) {
    tokens <- tokenize_text(text)
    build_ngram_table(tokens, n)
}
digest_url <- function(url, n) {
    res <- httr::GET(url)
    txt <- httr::content(res, as = "text", encoding = "UTF-8")
    digest_text(txt, n)
}
random_start <- function(tbl, sep = "\x1f") {
    keys <- ls(envir = tbl, all.names = TRUE)
    if (length(keys) == 0) stop("No n-grams available. Digest text first.")
    picked <- sample(keys, 1)
    strsplit(picked, sep, fixed = TRUE)[[1]]
}

predict_next_word <- function(tbl, ngram, sep = "\x1f") {
    key <- paste(ngram, collapse = sep)
    counts <- if (!is.null(tbl[[key]])) tbl[[key]] else integer(0)
    if (length(counts) == 0) return(NA_character_)
    sample(names(counts), size = 1, prob = as.numeric(counts))
}
make_ngram_generator <- function(tbl, n, sep = "\x1f") {
    force(tbl)
    n <- as.integer(n)
    force(sep)
 function(start_words = NULL, length = 10L) {
  if (is.null(start_words) || length(start_words) != n - 1L) {
            start_words <- random_start(tbl, sep = sep)
        }

        word_sequence <- start_words
  for (i in seq_len(max(0L, length - length(start_words)))) {
            ngram <- tail(word_sequence, n - 1L)
            next_word <- predict_next_word(tbl, ngram, sep = sep)
            if (is.na(next_word)) break
            word_sequence <- c(word_sequence, next_word)
        }

        paste(word_sequence, collapse = " ")
    }
}

cat("Building trigram model (n=3)...\n")
tbl3 <- digest_url(url, n = 3)
gen3 <- make_ngram_generator(tbl3, n = 3)

cat("\n\nWith start words 'the king':\n")
print(gen3(start_words = c("the", "king"), length = 15))

cat("\nRandom start, 15 words:\n")
print(gen3(length = 15))



Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



Building trigram model (n=3)...


With start words 'the king':
[1] "the king mourned over his three wishes so he took leave of each glass and"

Random start, 15 words:
[1] "beautiful garden and asked what was in every day she stole away and they have"


#### b) Test your model using a text file of [Ancient Armour and Weapons in Europe](https://www.gutenberg.org/cache/epub/46342/pg46342.txt)
#### i) Using n=3, with the start word(s) "the king", with length=15.
#### ii) Using n=3, with no start word, with length=15.

In [14]:
install.packages("tokenizers")
library(httr)
library(tokenizers)

url <- "https://www.gutenberg.org/cache/epub/46342/pg46342.txt"

tokenize_text <- function(text) {
    tokenizers::tokenize_words(text, lowercase = TRUE, strip_punct = TRUE)[[1]]
}

key_from <- function(ngram, sep = "\x1f") {
    paste(ngram, collapse = sep)
}


build_ngram_table <- function(tokens, n, sep = "\x1f") {
    if (length(tokens) < n) return(new.env(parent = emptyenv()))

    tbl <- new.env(parent = emptyenv())

    for (i in seq_len(length(tokens) - n + 1L)) {

        ngram <- tokens[i:(i + n - 2L)]

        next_word <- tokens[i + n - 1L]


        key <- key_from(ngram, sep)


        counts <- if (!is.null(tbl[[key]])) tbl[[key]] else integer(0)
        if (next_word %in% names(counts)) {
            counts[[next_word]] <- counts[[next_word]] + 1L
        } else {
            counts[[next_word]] <- 1L
        }
        tbl[[key]] <- counts
    }

    tbl
}
digest_text <- function(text, n) {
    tokens <- tokenize_text(text)
    build_ngram_table(tokens, n)
}
digest_url <- function(url, n) {
    res <- httr::GET(url)
    txt <- httr::content(res, as = "text", encoding = "UTF-8")
    digest_text(txt, n)
}
random_start <- function(tbl, sep = "\x1f") {
    keys <- ls(envir = tbl, all.names = TRUE)
    if (length(keys) == 0) stop("No n-grams available. Digest text first.")
    picked <- sample(keys, 1)
    strsplit(picked, sep, fixed = TRUE)[[1]]
}

predict_next_word <- function(tbl, ngram, sep = "\x1f") {
    key <- paste(ngram, collapse = sep)
    counts <- if (!is.null(tbl[[key]])) tbl[[key]] else integer(0)
    if (length(counts) == 0) return(NA_character_)
    sample(names(counts), size = 1, prob = as.numeric(counts))
}
make_ngram_generator <- function(tbl, n, sep = "\x1f") {
    force(tbl)
    n <- as.integer(n)
    force(sep)
 function(start_words = NULL, length = 10L) {
  if (is.null(start_words) || length(start_words) != n - 1L) {
            start_words <- random_start(tbl, sep = sep)
        }

        word_sequence <- start_words
  for (i in seq_len(max(0L, length - length(start_words)))) {
            ngram <- tail(word_sequence, n - 1L)
            next_word <- predict_next_word(tbl, ngram, sep = sep)
            if (is.na(next_word)) break
            word_sequence <- c(word_sequence, next_word)
        }

        paste(word_sequence, collapse = " ")
    }
}

cat("Building trigram model (n=3)...\n")
tbl3 <- digest_url(url, n = 3)
gen3 <- make_ngram_generator(tbl3, n = 3)

cat("\n\nWith start words 'the king':\n")
print(gen3(start_words = c("the", "king"), length = 15))

cat("\nRandom start, 15 words:\n")
print(gen3(length = 15))

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



Building trigram model (n=3)...


With start words 'the king':
[1] "the king wherever he may go in order to be chain mail or of quilted"

Random start, 15 words:
[1] "engraved excellently and in the eleventh century for of raoul de beaumont in the archæologia"


#### c) Explain in 1-2 sentences the difference in content generated from each source.

The main difference is the choice of words. For the second random start, the machine picked something with lots of numbers and _lib which I am not sure what that means. For the first example the machine chose words that were more coherent.

## Question 3
#### a) What is a language learning model?
A type of AI that is trained with lots of text to be able to generate human-like language

#### b) Imagine the internet goes down and you can't run to your favorite language model for help. How do you run one locally?

To run locally you can get OLLAMA installed you can use this and write your own code allowing you to run your favorite language model locally.

## Question 4
#### Explain what the following vocab words mean in the context of typing `mkdir project` into the command line. If the term doesn't apply to this command, give the definition and/or an example.
| Term | Meaning |  
|------|---------|
| **Shell** | Executes and interprets mkdir. It runs the command and passes project as an object to the command |
| **Terminal emulator** | Graphical application window. For example for mac we have the terminal. This is where you type mkdir project into |
| **Process** | When you run mkdir project and the os creates a new system. This process has its own memory space and runs independantly |
| **Signal** | mkdir project doesn't direclty use it but if you were to crl c while mkdir was running, there would be a signal sent to terminate program |
| **Standard input** | Standard input is where a program reads data from - typically keyboard input or piped data. |
| **Standard output** | Standard output is where programs write their results. |
| **Command line argument** | Project is a command line argument passed through mkdir. These arguments are inputs that are provided after a command |
| **The environment** | all the stuff a process can see when its running; ok to imagine that when one process starts another, the child process sees all the same stuff as the parent  |

## Question 5
#### Consider the following command `find . -iname "*.R" | xargs grep read_csv`.
#### a) What are the programs?
find, xargs, grep
#### b) Explain what this command is doing, part by part.

find parses through the files, the . is the starting directory, the -iname "*.R" means its searching for files that have *.R and matches files that end in R or r. The pipe operator takes the standard output from find and sends it as standard input to the next command, the xargs Reads filenames from the input and converts them into arguments for the next command. grep makes the program that will search for patterns in files. The read_csv is what the grep will be looking for when its looking through the lines of code.

## Question 6
#### Install Docker on your machine. See [here](https://github.com/rjenki/BIOS512/blob/main/lecture18/docker_install.md) for instructions.
#### a) Show the response when you run `docker run hello-world`.

Unable to find image 'hello-world:latest' locally
latest: Pulling from library/hello-world
198f93fd5094: Pull complete
Digest: sha256:f7931603f70e13dbd844253370742c4fc4202d290c80442b2e68706d8f33ce26
Status: Downloaded newer image for hello-world:latest

Hello from Docker!
This message shows that your installation appears to be working correctly.

To generate this message, Docker took the following steps:
 1. The Docker client contacted the Docker daemon.
 2. The Docker daemon pulled the "hello-world" image from the Docker Hub.
    (arm64v8)
 3. The Docker daemon created a new container from that image which runs the
    executable that produces the output you are currently reading.
 4. The Docker daemon streamed that output to the Docker client, which sent it
    to your terminal.

To try something more ambitious, you can run an Ubuntu container with:
 $ docker run -it ubuntu bash

Share images, automate workflows, and more with a free Docker ID:
 https://hub.docker.com/

For more examples and ideas, visit:
 https://docs.docker.com/get-started/

#### b) Access Rstudio through a Docker container. Set your password and make sure your files show up on the Rstudio server. Type the command and the output you get below.
docker pull rocker/rstudio
docker run -d -p 8787:8787 -e PASSWORD=yourpassword rocker/rstudio

Using default tag: latest
latest: Pulling from rocker/rstudio
08e74fd5985d: Pull complete
e2804bef35e8: Pull complete
91ed5b86de88: Pull complete
cc9c938c1f51: Pull complete
bcce866b1806: Pull complete
5b219f62ce36: Pull complete
2034506aa72f: Pull complete
9c1a4a0706b7: Pull complete
5d246ec925db: Pull complete
bc9245ceaac5: Pull complete
b8a35db46e38: Pull complete
3665120d345d: Pull complete
a730ff463d58: Pull complete
2c9ba66d5dbe: Pull complete
39038e16d1ba: Pull complete
664fb1818bbb: Pull complete
191985778909: Pull complete
abd0190d83fb: Pull complete
Digest: sha256:9f85211a666fb426081a6f5a01f9f9f51655262258419fa21e0ce38a5afc78d8
Status: Downloaded newer image for rocker/rstudio:latest
docker.io/rocker/rstudio:latest

#### c) How do you log in to the RStudio server?

You have a local host, and it allows you to login and do continue your coding. The username is rstudio by defualt and the password is whatever I put it.  